# Import packages and data


In [ ]:
import numpy as np
import pandas as pd
import sys
import pickle
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/456/submit_code


# Data Cleaning and transfer ICD9 to CCS encode


## Admissions table cleaning

In [ ]:
older15patients = pd.read_csv('/content/drive/MyDrive/456/submit_code/data/older15patients.csv') # the table with age>15, selected using SQL
admissions = pd.read_csv('/content/drive/MyDrive/456/submit_code/data/ADMISSIONS.csv')
procedures = pd.read_csv('/content/drive/MyDrive/456/submit_code/data/PROCEDURES_ICD.csv')
diagnoses = pd.read_csv('/content/drive/MyDrive/456/submit_code/data/DIAGNOSES_ICD.csv')

In [ ]:
# only keep older than 15 pacients' admissions
admissions = admissions[admissions['HADM_ID'].isin(older15patients['HADM_ID'])]
admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
15,36,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
16,37,36,122659,2131-05-12 19:49:00,2131-05-25 13:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,2131-05-12 17:26:00,2131-05-12 22:17:00,CHEST PAIN/SHORTNESS OF BREATH,0,1
17,38,36,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,VENTRAL HERNIA/SDA,0,1


In [ ]:
diagnoses.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [ ]:
procedures.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [ ]:
len(diagnoses['ICD9_CODE'].unique()) # Too many classes for classification

6985

In [ ]:
# Exclude patients who only had one occurrence [where the frequency of SUBJECT_ID > 1]
subject_id_counts = admissions['SUBJECT_ID'].value_counts()
single_occurrence_subject_ids = subject_id_counts[subject_id_counts == 1].index
adm = admissions[~admissions['SUBJECT_ID'].isin(single_occurrence_subject_ids)]
adm.to_csv('/content/drive/MyDrive/456/submit_code/data/admissions_cleaned_15.csv', index=False)
adm.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
15,36,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
16,37,36,122659,2131-05-12 19:49:00,2131-05-25 13:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,2131-05-12 17:26:00,2131-05-12 22:17:00,CHEST PAIN/SHORTNESS OF BREATH,0,1
17,38,36,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,VENTRAL HERNIA/SDA,0,1


## Diagnoses and Procedures Tables: Change ICD-9 code to CCS code

In [ ]:
# Import the transfer table and change the form of data inside
ccs_dx = pd.read_csv('ccs_multi_dx_tool_2015.csv')
ccs_dx.columns = ['ICD-9-CM CODE', 'CCS LVL 1', 'CCS LVL 1 LABEL', 'CCS LVL 2', 'CCS LVL 2 LABEL', 'CCS LVL 3', 'CCS LVL 3 LABEL','CCS LVL 4','CCS LVL 4 LABEL']
ccs_dx['ICD-9-CM CODE'] = ccs_dx['ICD-9-CM CODE'].str.strip("'")
ccs_dx['CCS LVL 2'] = ccs_dx['CCS LVL 2'].str.strip("'")
ccs_dx['ICD-9-CM CODE'] = ccs_dx['ICD-9-CM CODE'].str.strip()
ccs_dx.head()

,ICD-9-CM CODE,CCS LVL 1,CCS LVL 1 LABEL,CCS LVL 2,CCS LVL 2 LABEL,CCS LVL 3,CCS LVL 3 LABEL,CCS LVL 4,CCS LVL 4 LABEL
0,01000,'1',Infectious and parasitic diseases,1.1,Bacterial infection,'1.1.1',Tuberculosis [1.],' ',
1,01001,'1',Infectious and parasitic diseases,1.1,Bacterial infection,'1.1.1',Tuberculosis [1.],' ',
2,01002,'1',Infectious and parasitic diseases,1.1,Bacterial infection,'1.1.1',Tuberculosis [1.],' ',
3,01003,'1',Infectious and parasitic diseases,1.1,Bacterial infection,'1.1.1',Tuberculosis [1.],' ',
4,01004,'1',Infectious and parasitic diseases,1.1,Bacterial infection,'1.1.1',Tuberculosis [1.],' ',


In [ ]:
# Dropout null values and change to strings
diagnoses = diagnoses.dropna(subset=['ICD9_CODE'])
diagnoses['ICD9_CODE'] = diagnoses['ICD9_CODE'].astype(str)
# Merge tables A and B based on 'ICD9_CODE' and 'ICD-9-CM CODE'
merged_df_dx = pd.merge(diagnoses, ccs_dx, left_on='ICD9_CODE', right_on='ICD-9-CM CODE', how='left')

<ipython-input-14-ecc4d3f5ee8b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnoses['ICD9_CODE'] = diagnoses['ICD9_CODE'].astype(str)


In [ ]:
# Change column names
diagnoses_label= merged_df_dx[['ROW_ID',	'SUBJECT_ID',	'HADM_ID',	'SEQ_NUM',	'ICD9_CODE',	'CCS LVL 2']]
diagnoses_label.columns = ['ROW_ID',	'SUBJECT_ID',	'HADM_ID',	'SEQ_NUM',	'ICD9_CODE',	'label']
diagnoses_label.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,label
0,1297,109,172335,1.0,40301,7.1
1,1298,109,172335,2.0,486,8.1
2,1299,109,172335,3.0,58281,10.1
3,1300,109,172335,4.0,5855,10.1
4,1301,109,172335,5.0,4254,7.2


In [ ]:
diagnoses_label.to_csv('/content/drive/MyDrive/456/submit_code/data/diagnoses_label.csv', index=False)

In [ ]:
# Same action as before
ccs_pr = pd.read_csv('ccs_multi_pr_tool_2015.csv')
ccs_pr.columns = ['ICD-9-CM CODE', 'CCS LVL 1', 'CCS LVL 1 LABEL', 'CCS LVL 2', 'CCS LVL 2 LABEL', 'CCS LVL 3', 'CCS LVL 3 LABEL']
ccs_pr['ICD-9-CM CODE'] = ccs_pr['ICD-9-CM CODE'].str.strip("'")
ccs_pr['ICD-9-CM CODE'] = ccs_pr['ICD-9-CM CODE'].str.strip()
ccs_pr['CCS LVL 2'] = ccs_pr['CCS LVL 2'].str.strip("'")
ccs_pr.head()

,ICD-9-CM CODE,CCS LVL 1,CCS LVL 1 LABEL,CCS LVL 2,CCS LVL 2 LABEL,CCS LVL 3,CCS LVL 3 LABEL
0,0121,'1',Operations on the nervous system,1.1,Incision and excision of CNS [1.],'1.1.1',Craniotomy and craniectomy
1,0122,'1',Operations on the nervous system,1.1,Incision and excision of CNS [1.],'1.1.1',Craniotomy and craniectomy
2,0123,'1',Operations on the nervous system,1.1,Incision and excision of CNS [1.],'1.1.1',Craniotomy and craniectomy
3,0124,'1',Operations on the nervous system,1.1,Incision and excision of CNS [1.],'1.1.1',Craniotomy and craniectomy
4,0125,'1',Operations on the nervous system,1.1,Incision and excision of CNS [1.],'1.1.1',Craniotomy and craniectomy


In [ ]:
procedures['ICD9_CODE'] = procedures['ICD9_CODE'].astype(str)
merged_df_pr = pd.merge(procedures, ccs_pr, left_on='ICD9_CODE', right_on='ICD-9-CM CODE', how='left')

# Select the 'CCS LVL 2 LABEL' column as the new 'label' column in table A
A = procedures
A['label'] = merged_df_pr['CCS LVL 2']

# change all the ICD-9 code to 4 digits
B = A[A['label'].isnull()]
B['ICD9_CODE'] = B['ICD9_CODE'].astype(str).str.zfill(4)
B = B[['HADM_ID','ROW_ID','SUBJECT_ID','SEQ_NUM','ICD9_CODE']]
merged_df_pr_2 = pd.merge(B, ccs_pr, left_on='ICD9_CODE', right_on='ICD-9-CM CODE', how='left')

B = merged_df_pr_2[['SUBJECT_ID','HADM_ID','ROW_ID','SEQ_NUM','ICD9_CODE','CCS LVL 2']]
B.columns = ['SUBJECT_ID','HADM_ID','ROW_ID','SEQ_NUM','ICD9_CODE','label']
B = B[['ROW_ID',	'SUBJECT_ID',	'HADM_ID',	'SEQ_NUM',	'ICD9_CODE',	'label']]
C = A[A['label'].isnull()==False]
procedures_label = pd.concat([B, C])
procedures_label = procedures_label.sort_values(by='ROW_ID')
procedures_label.head()

<ipython-input-21-5b87c092333a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B['ICD9_CODE'] = B['ICD9_CODE'].astype(str).str.zfill(4)


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,label
422,1,9584,151156,1,5123,9.16
423,2,9584,151156,2,5187,9.30
424,3,9584,151156,3,5185,9.30
425,4,21274,175983,1,3893,7.12
426,5,21274,175983,2,8961,16.38


In [ ]:
procedures_label.to_csv('/content/drive/MyDrive/456/submit_code/data/procedures_label.csv', index=False)

# Batch processing

In [ ]:
# This script processes MIMIC-III dataset and builds longitudinal diagnosis records for patients with at least two visits.
# The output data are cPickled, and suitable for training model
# Output files
# <output file>.pids: List of unique Patient IDs. Used for intermediate processing
# <output file>.dates: List of List of Python datetime objects. The outer List is for each patient. The inner List is for each visit made by each patient
# <output file>.seqs: List of List of List of integer diagnosis codes. The outer List is for each patient. The middle List contains visits made by each patient. The inner List contains the integer diagnosis codes that occurred in each visit
# <output file>.types: Python dictionary that maps string diagnosis codes to integer diagnosis codes.

In [ ]:
# diagnose data
admissionFile = "/content/drive/MyDrive/456/submit_code/data/admissions_cleaned_15.csv"
diagnosisFile = "/content/drive/MyDrive/456/submit_code/data/diagnoses_label.csv"
diagnosisoutFile = "/content/drive/MyDrive/456/submit_code/data/diagnose"

In [ ]:
# procedure data
admissionFile = "/content/drive/MyDrive/456/submit_code/data/admissions_cleaned_15.csv"
procedureFile = "/content/drive/MyDrive/456/submit_code/data/procedures_label.csv"
procedureoutFile = "/content/drive/MyDrive/456/submit_code/data/procedures"

In [ ]:
def data_process(admissionFile, eventFile, outFile):
  print ('Building pid-admission mapping, admission-date mapping')
  pidAdmMap = {}
  admDateMap = {}
  infd = open(admissionFile, 'r')
  infd.readline()
  for line in infd:
    tokens = line.strip().split(',')
    pid = int(tokens[1])
    admId = int(tokens[2])
    admTime = datetime.strptime(tokens[3], '%Y-%m-%d %H:%M:%S')
    admDateMap[admId] = admTime
    if pid in pidAdmMap: pidAdmMap[pid].append(admId)
    else: pidAdmMap[pid] = [admId]
  infd.close()

  print ('Building admission-dxList mapping')
  admDxMap = {}
  infd = open(diagnosisFile, 'r')
  infd.readline()
  for line in infd:
    tokens = line.strip().split(',')
    admId = int(tokens[2])
    dxStr = 'D_' + tokens[5] #4:ICD9, 5:CCS
    if admId in admDxMap: admDxMap[admId].append(dxStr)
    else: admDxMap[admId] = [dxStr]
  infd.close()

  print('Building pid-sortedVisits mapping')
  pidSeqMap = {}
  for pid, admIdList in pidAdmMap.items():
    if len(admIdList) < 2: continue
    sortedList = sorted([(admDateMap[admId], admDxMap[admId]) for admId in admIdList])
    pidSeqMap[pid] = sortedList

  print ('Building pids, dates, strSeqs')
  pids = []
  dates = []
  seqs = []
  for pid, visits in pidSeqMap.items():
    pids.append(pid)
    seq = []
    date = []
    for visit in visits:
      date.append(visit[0])
      seq.append(visit[1])
    dates.append(date)
    seqs.append(seq)

  print ('Converting strSeqs to intSeqs, and making types')
  types = {}
  newSeqs = []
  for patient in seqs:
    newPatient = []
    for visit in patient:
      newVisit = []
      for code in visit:
        if code in types:
          newVisit.append(types[code])
        else:
          types[code] = len(types)
          newVisit.append(types[code])
      newPatient.append(newVisit)
    newSeqs.append(newPatient)

  pickle.dump(pids, open(outFile+'.pids', 'wb'), -1)
  pickle.dump(dates, open(outFile+'.dates', 'wb'), -1)
  pickle.dump(newSeqs, open(outFile+'.seqs', 'wb'), -1)
  pickle.dump(types, open(outFile+'.types', 'wb'), -1)

In [ ]:
data_process(admissionFile, diagnosisFile, diagnosisoutFile)
data_process(admissionFile, procedureFile, procedureoutFile)

Building pid-admission mapping, admission-date mapping
Building admission-dxList mapping
Building pid-sortedVisits mapping
Building pids, dates, strSeqs
Converting strSeqs to intSeqs, and making types
Building pid-admission mapping, admission-date mapping
Building admission-dxList mapping
Building pid-sortedVisits mapping
Building pids, dates, strSeqs
Converting strSeqs to intSeqs, and making types
